refactoring of the initial cleaning i had done but without importing data into flosp

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import flosp
#import bradlib as bl
#sns.set()

In [2]:
df  = pd.read_csv('./../../../../3_Data/EDclassification/WAC_UHS_EDexport_0918_rr.txt',low_memory=False)

ed = flosp.ioED('sgh','./../../3_Data/')

ed.load_csv('./../../3_Data/WAC_UHS_EDexport_0918_rr.txt')

map clean cols

In [3]:
df.columns

Index(['Pseudo Patient Number', 'ATTENDANCE_NUMBER', 'Age', 'Gender',
       'Department', 'Site attended', 'Arrival', 'Arrival mode',
       'Arrival mode desc', '1st time triaged [datetime]',
       '1st time seen by doctor', 'Time of inpatient bed request',
       'Time of speciality referral', 'Speciality referred to',
       'Time into CDU', 'Time leaving dept (inc CDU)', 'Admission flag',
       'Majors/Minors/Resus patient', 'Triage category'],
      dtype='object')

In [4]:
df.head(2)

,Pseudo Patient Number,ATTENDANCE_NUMBER,Age,Gender,Department,Site attended,Arrival,Arrival mode,Arrival mode desc,1st time triaged [datetime],1st time seen by doctor,Time of inpatient bed request,Time of speciality referral,Speciality referred to,Time into CDU,Time leaving dept (inc CDU),Admission flag,Majors/Minors/Resus patient,Triage category
0,004101045105,90902325901,94.0,F,Main ED,Main ED,13-APR-09 09:16,NaN,BROUGHT IN BY AMBULANCE,NaN,13-APR-09 10:28,NaN,NaN,NaN,NaN,13-APR-09 12:24,N,Majors,VERY URGENT
1,644868441371,90902326101,47.0,M,Main ED,Main ED,13-APR-09 09:22,NaN,BROUGHT IN BY AMBULANCE,NaN,13-APR-09 09:30,NaN,NaN,NaN,NaN,13-APR-09 12:55,Y,Majors,VERY URGENT


In [5]:
df.tail(2)

,Pseudo Patient Number,ATTENDANCE_NUMBER,Age,Gender,Department,Site attended,Arrival,Arrival mode,Arrival mode desc,1st time triaged [datetime],1st time seen by doctor,Time of inpatient bed request,Time of speciality referral,Speciality referred to,Time into CDU,Time leaving dept (inc CDU),Admission flag,Majors/Minors/Resus patient,Triage category
1124685,206828068058,91734736,44.0,F,Eye Casualty,Eye Casualty,09-MAY-18 08:24,NaN,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,09-MAY-18 10:18,N,NaN,UNKNOWN
1124686,854787547747,91804267301,37.0,M,Main ED,Main ED,27-MAY-18 00:13,Emergency Road Ambulance,BROUGHT IN BY AMBULANCE,NaN,NaN,27-MAY-18 04:38,NaN,NaN,27-MAY-18 04:43,27-MAY-18 10:04,N,NaN,STANDARD


Clean col names

In [6]:
from flosp.basic_tools import tidy_column_heads

In [7]:
df = tidy_column_heads(df)

Exclude eye cashualty

In [8]:
df.site_attended.value_counts()

Main ED               907932
Eye Casualty          210850
GP Urgent Care Hub      4940
Cath Lab                 548
Bramshaw                 417
Name: site_attended, dtype: int64

In [9]:
df.shape

(1124687, 19)

In [10]:
df = df.query('site_attended != "Eye Casualty"')
df.shape

(913837, 19)

whats missing?

In [11]:
df.isnull().sum()

pseudo_patient_number                 1
attendance_number                     0
age                                  10
gender                               10
department                            0
site_attended                         0
arrival                               0
arrival_mode                     269946
arrival_mode_desc                     0
1st_time_triaged_[datetime]      280630
1st_time_seen_by_doctor           76978
time_of_inpatient_bed_request    658458
time_of_speciality_referral      653702
speciality_referred_to           653703
time_into_cdu                    816058
time_leaving_dept_inc_cdu             0
admission_flag                        0
majorsminorsresus_patient        114766
triage_category                  639560
dtype: int64

# commence cleaning

#### define dictionary for mapping
col_map = {
'ATTENDANCE_NUMBER':'dept_patid',
'Pseudo Patient Number':'hosp_patid',
'Age':'age',
'Gender':'gender',
'Site attended':'site',
'Arrival':'arrive_datetime',
'Arrival mode':'arrive_mode',
'1st time triaged [datetime]':'first_triage_datetime',
'1st time seen by doctor':'first_dr_datetime',
'Time of inpatient bed request':'first_adm_request_datetime',
'Speciality referred to':'adm_referral_loc',
'Admission flag':'adm_flag',
'Time leaving dept (inc CDU)':'depart_datetime',
'Majors/Minors/Resus patient':'stream'
}

ed.map_columns(col_map)

ed._dataRAW.head(2)

ed._dataRAW['site'].value_counts()

Huge numbers are missing departure times?!'  -half?

ed._dataRAW.shape

ed._dataRAW[ed._dataRAW['depart_datetime'].isnull()].head(1)

#### clean adm flag

In [12]:
df.columns

Index(['pseudo_patient_number', 'attendance_number', 'age', 'gender',
       'department', 'site_attended', 'arrival', 'arrival_mode',
       'arrival_mode_desc', '1st_time_triaged_[datetime]',
       '1st_time_seen_by_doctor', 'time_of_inpatient_bed_request',
       'time_of_speciality_referral', 'speciality_referred_to',
       'time_into_cdu', 'time_leaving_dept_inc_cdu', 'admission_flag',
       'majorsminorsresus_patient', 'triage_category'],
      dtype='object')

In [13]:
z = {'Y':1,'N':0}
df['flag_admission'] = df.admission_flag.map(z)

#### clean gender

In [14]:
df.gender.value_counts()

M    465247
F    448551
U        27
I         2
Name: gender, dtype: int64

In [15]:
z = {'M':1,'F':0,'U':0,'I':0}
df['gender'] = df.gender.map(z)

In [16]:
# df['gender'] = 
df.gender.fillna(1,inplace=True)

In [17]:
df.gender.value_counts()

1.0    465257
0.0    448580
Name: gender, dtype: int64

In [18]:
df.gender.isnull().sum()

0

In [19]:
df['gender'] = df.gender.astype('int32')

In [20]:
df.gender.value_counts()

1    465257
0    448580
Name: gender, dtype: int64

### clean age

In [21]:
df.age.isnull().sum()

10

In [22]:
median = df.age.median()

In [23]:
df.age.fillna(value=median, inplace=True)

In [24]:
df['age'] = df.age.astype(int)

#### flag patient had bed request/speciality req/entered cdu

In [25]:
df['flag_bedreq'] = df.time_of_inpatient_bed_request.isnull().astype(int)

In [26]:
df['flag_specreq'] = df.time_of_speciality_referral.isnull().astype(int)

In [27]:
df['flag_cdu'] = df.time_into_cdu.isnull().astype(int)

In [28]:
df.flag_admission.value_counts()

0    559945
1    353892
Name: flag_admission, dtype: int64

Note: more missing adm_flag values for adm_referral_loc, have used this to mark additional adm_flags. addiitonal 30k records ups coversion ratio by 2%...see below (albeit post assingment).

#### covert datetime cols

In [29]:
df.columns

Index(['pseudo_patient_number', 'attendance_number', 'age', 'gender',
       'department', 'site_attended', 'arrival', 'arrival_mode',
       'arrival_mode_desc', '1st_time_triaged_[datetime]',
       '1st_time_seen_by_doctor', 'time_of_inpatient_bed_request',
       'time_of_speciality_referral', 'speciality_referred_to',
       'time_into_cdu', 'time_leaving_dept_inc_cdu', 'admission_flag',
       'majorsminorsresus_patient', 'triage_category', 'flag_admission',
       'flag_bedreq', 'flag_specreq', 'flag_cdu'],
      dtype='object')

In [30]:
dt_cols = ['arrival','time_leaving_dept_inc_cdu','time_of_inpatient_bed_request','time_into_cdu','time_of_speciality_referral','1st_time_seen_by_doctor']

for i in dt_cols:
    df[i] = pd.to_datetime(df[i], format= '%d-%b-%y %H:%M')

### tidy and rename cols

In [31]:
df.head(2)

,pseudo_patient_number,attendance_number,age,gender,department,site_attended,arrival,arrival_mode,arrival_mode_desc,1st_time_triaged_[datetime],...,speciality_referred_to,time_into_cdu,time_leaving_dept_inc_cdu,admission_flag,majorsminorsresus_patient,triage_category,flag_admission,flag_bedreq,flag_specreq,flag_cdu
0,004101045105,90902325901,94,0,Main ED,Main ED,2009-04-13 09:16:00,NaN,BROUGHT IN BY AMBULANCE,NaN,...,NaN,NaT,2009-04-13 12:24:00,N,Majors,VERY URGENT,0,1,1,1
1,644868441371,90902326101,47,1,Main ED,Main ED,2009-04-13 09:22:00,NaN,BROUGHT IN BY AMBULANCE,NaN,...,NaN,NaT,2009-04-13 12:55:00,Y,Majors,VERY URGENT,1,1,1,1


In [32]:
df.department.value_counts()

Main ED    913837
Name: department, dtype: int64

In [33]:
df.site_attended.value_counts()

Main ED               907932
GP Urgent Care Hub      4940
Cath Lab                 548
Bramshaw                 417
Name: site_attended, dtype: int64

In [34]:
df = df.drop(labels = ['department','admission_flag'], axis=1)

In [35]:
df.isnull().sum()

pseudo_patient_number                 1
attendance_number                     0
age                                   0
gender                                0
site_attended                         0
arrival                               0
arrival_mode                     269946
arrival_mode_desc                     0
1st_time_triaged_[datetime]      280630
1st_time_seen_by_doctor           76978
time_of_inpatient_bed_request    658458
time_of_speciality_referral      653702
speciality_referred_to           653703
time_into_cdu                    816058
time_leaving_dept_inc_cdu             0
majorsminorsresus_patient        114766
triage_category                  639560
flag_admission                        0
flag_bedreq                           0
flag_specreq                          0
flag_cdu                              0
dtype: int64

In [36]:
df.columns

Index(['pseudo_patient_number', 'attendance_number', 'age', 'gender',
       'site_attended', 'arrival', 'arrival_mode', 'arrival_mode_desc',
       '1st_time_triaged_[datetime]', '1st_time_seen_by_doctor',
       'time_of_inpatient_bed_request', 'time_of_speciality_referral',
       'speciality_referred_to', 'time_into_cdu', 'time_leaving_dept_inc_cdu',
       'majorsminorsresus_patient', 'triage_category', 'flag_admission',
       'flag_bedreq', 'flag_specreq', 'flag_cdu'],
      dtype='object')

In [37]:
cols_rename = {
    'arrival':'arrive_datetime',
    'time_leaving_dept_inc_cdu':'depart_datetime',
}
df.rename(columns=cols_rename, inplace=True)

# save out

In [38]:
df.head()

,pseudo_patient_number,attendance_number,age,gender,site_attended,arrive_datetime,arrival_mode,arrival_mode_desc,1st_time_triaged_[datetime],1st_time_seen_by_doctor,...,time_of_speciality_referral,speciality_referred_to,time_into_cdu,depart_datetime,majorsminorsresus_patient,triage_category,flag_admission,flag_bedreq,flag_specreq,flag_cdu
0,004101045105,90902325901,94,0,Main ED,2009-04-13 09:16:00,NaN,BROUGHT IN BY AMBULANCE,NaN,2009-04-13 10:28:00,...,NaT,NaN,NaT,2009-04-13 12:24:00,Majors,VERY URGENT,0,1,1,1
1,644868441371,90902326101,47,1,Main ED,2009-04-13 09:22:00,NaN,BROUGHT IN BY AMBULANCE,NaN,2009-04-13 09:30:00,...,NaT,NaN,NaT,2009-04-13 12:55:00,Majors,VERY URGENT,1,1,1,1
2,668666687357,90902326501,0,1,Main ED,2009-04-13 09:31:00,NaN,OTHER,NaN,2009-04-13 10:00:00,...,2009-04-13 11:19:00,Paediatrics - Medical,NaT,2009-04-13 11:54:00,Paeds,STANDARD,1,1,0,1
3,446747461391,90902328601,4,1,Main ED,2009-04-13 10:52:00,NaN,OTHER,NaN,2009-04-13 11:30:00,...,2009-04-13 12:09:00,Orthopaedics,NaT,2009-04-13 13:49:00,NaN,STANDARD,0,1,0,1
4,423646238168,90902328801,31,0,Main ED,2009-04-13 11:18:00,NaN,BROUGHT IN BY AMBULANCE,NaN,2009-04-13 12:00:00,...,2009-04-13 12:27:00,Obstetrics & Gynaecology,NaT,2009-04-13 14:49:00,Majors,VERY URGENT,1,1,0,1


In [39]:
df.to_csv('./../../../../3_Data/EDclassification/cleanED.csv', index=False)

In [40]:
df.to_pickle('./../../../../3_Data/EDclassification/cleanED.pkl')

# DEV

basic plotting for checks

In [30]:
break

SyntaxError: 'break' outside loop (<ipython-input-30-6aaf1f276005>, line 1)